In [103]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [104]:
# 参数设置
theta_deg = 120  # 开角（度）
alpha_deg = 1.5  # 坡度（度）
# beta_deg = 0  # 侧向角（度）
beta_list = np.arange(0, 360, 45)  # 侧向角（度），从0到360度，每45度一个值
D0 = 120  # 中心深度（米）
d = 0.3  # 测线间距（海里）
d_range_ = np.arange(0, 2.2, d)  # 沿测线距离（海里）
d_range = d_range_ * 1852  # 1 海里 = 1852 米
print(
    f"参数设置:\n开角 θ: {theta_deg}°\n"
    f"坡度 α: {alpha_deg}°\n"
    f"侧向角 β: {beta_list}°\n"
    f"中心深度 D0: {D0} 米\n"
    f"测线间距 d: {d} 海里\n"
    f"沿测线距离范围: {d_range} 米"
)

参数设置:
开角 θ: 120°
坡度 α: 1.5°
侧向角 β: [  0  45  90 135 180 225 270 315]°
中心深度 D0: 120 米
测线间距 d: 0.3 海里
沿测线距离范围: [   0.   555.6 1111.2 1666.8 2222.4 2778.  3333.6 3889.2] 米


In [105]:
# 转换为弧度
theta = np.radians(theta_deg)
beta_list = np.radians(beta_list)  # 侧向角（弧度）
# beta = np.radians(beta_deg)
alpha = np.radians(alpha_deg)

In [106]:
results = []

for beta in beta_list:
    # 坡度在当前方向的有效分量
    alpha_ = np.arcsin(np.sin(alpha) * np.cos(beta))
    print(f"\n计算侧向角 β = {np.degrees(beta)}° 时的坡度 α' = {np.degrees(alpha_)}°")
    D = D0 + d_range * np.tan(alpha_)
    print(f"计算得到的深度 D:\n{D}")
    W_left = D * np.sin(theta / 2) / np.cos(theta / 2 + alpha_)
    W_right = D * np.sin(theta / 2) / np.cos(theta / 2 - alpha_)
    W_total = W_left + W_right
    # print(f"计算得到的覆盖宽度 W:\n{W_total}")
    
    df_beta = pd.DataFrame({
        'beta': np.degrees(beta).astype(int),  # 将弧度转换为整数度数
        'd_range': d_range_,
		'D': D,
		# 'W_left': W_left,
		# 'W_right': W_right,
		'W_total': W_total,
	})
    print(df_beta)
    results.append(df_beta)
    


计算侧向角 β = 0.0° 时的坡度 α' = 1.5000000000000002°
计算得到的深度 D:
[120.         134.54889802 149.09779605 163.64669407 178.1955921
 192.74449012 207.29338814 221.84228617]
   beta  d_range           D     W_total
0     0      0.0  120.000000  416.691870
1     0      0.3  134.548898  467.211933
2     0      0.6  149.097796  517.731995
3     0      0.9  163.646694  568.252058
4     0      1.2  178.195592  618.772121
5     0      1.5  192.744490  669.292183
6     0      1.8  207.293388  719.812246
7     0      2.1  221.842286  770.332309

计算侧向角 β = 45.0° 时的坡度 α' = 1.0605995840948033°
计算得到的深度 D:
[120.         130.28586134 140.57172268 150.85758403 161.14344537
 171.42930671 181.71516805 192.0010294 ]
   beta  d_range           D     W_total
0    45      0.0  120.000000  416.191352
1    45      0.3  130.285861  451.865407
2    45      0.6  140.571723  487.539461
3    45      0.9  150.857584  523.213516
4    45      1.2  161.143445  558.887570
5    45      1.5  171.429307  594.561624
6    45      1.8

In [107]:
df_all = pd.concat(results, ignore_index=True)
# 保证 d_range 是 float
df_all['d_range'] = df_all['d_range'].astype(float)
# 保留一位小数（注意：建议用 round 再用 astype 保证不会有浮点数精度误差导致的 0.30000000004 这类情况）
df_all['d_range'] = df_all['d_range'].round(1)
df_all

,beta,d_range,D,W_total
0,0,0.0,120.000000,416.691870
1,0,0.3,134.548898,467.211933
2,0,0.6,149.097796,517.731995
3,0,0.9,163.646694,568.252058
4,0,1.2,178.195592,618.772121
...,...,...,...,...
59,315,0.9,150.857584,523.213516
60,315,1.2,161.143445,558.887570
61,315,1.5,171.429307,594.561624
62,315,1.8,181.715168,630.235679


In [108]:
df_pivot = df_all.pivot(index='beta', columns='d_range', values='W_total')
df_pivot.columns = [f"{c:.1f}" for c in df_pivot.columns]
df_pivot.columns.name = "d_range (海里)"
df_pivot.index.name = "测线方向夹角 β/°"
df_pivot.style.format("{:.2f}")

d_range (海里),0.0,0.3,0.6,0.9,1.2,1.5,1.8,2.1
测线方向夹角 β/°,,,,,,,,
0,416.69,467.21,517.73,568.25,618.77,669.29,719.81,770.33
45,416.19,451.87,487.54,523.21,558.89,594.56,630.24,665.91
90,415.69,415.69,415.69,415.69,415.69,415.69,415.69,415.69
135,416.19,380.52,344.84,309.17,273.50,237.82,202.15,166.47
180,416.69,366.17,315.65,265.13,214.61,164.09,113.57,63.05
225,416.19,380.52,344.84,309.17,273.50,237.82,202.15,166.47
270,415.69,415.69,415.69,415.69,415.69,415.69,415.69,415.69
315,416.19,451.87,487.54,523.21,558.89,594.56,630.24,665.91


In [109]:
# # 计算重叠率
# # 对于每对相邻测线（x_i, x_{i+1}），第1条的右宽度+第2条的左宽度
# W1r = W_right[:-1]    # 第i条右宽度
# W2l = W_left[1:]      # 第i+1条左宽度
# overlap = 1 - d / ((W1r + W2l) * np.cos(alpha))  # 重叠率

# print(f"计算得到的重叠率:\n{overlap}")